In [1]:
import csv
import numpy as np
xtrain_glove = np.loadtxt('query_glove.csv',delimiter=",", dtype=float)

from tqdm import tqdm


# Reading 2048-d train features 
features_train_2048 = np.zeros((10000,2048))
with open('./all/data/features_train/features_resnet1000intermediate_train.csv', 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    for line in csv_reader:
        image_name = line[0].strip(".jpg")[13:]
        row = []
        for i in range(len(line)):
            if i > 0:
                row.append(float(line[i]))
        features_train_2048[int(image_name)] = row
        
# Reading 1000-d train features 
features_train_1000 = np.zeros((10000,1000))
with open('./all/data/features_train/features_resnet1000_train.csv', 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    for line in csv_reader:
        image_name = line[0].strip(".jpg")[13:]
        row = []
        for i in range(len(line)):
            if i > 0:
                row.append(float(line[i]))
        features_train_1000[int(image_name)] = row
        
# Find set of supercategories, categories 
supercategory_set = set()
category_set = set()
for i in range (10000):
    file = open("./all/data/tags_train/" + str(i) + ".txt", "r")
    lines = file.readlines() 
    for  line in lines:
        words = line.strip().split(':')
        supercategory_set.add(words[0])
        category_set.add(words[1])
    file.close()

# mapping from (super) category to index
supercategory_dict = {item:val for val, item in enumerate(supercategory_set)}
category_dict = {item:val+1 for val, item in enumerate(category_set)}

# Vectorize train tags
train_tags = []
for i in range (10000):
    file = open("./all/data/tags_train/" + str(i) + ".txt", "r")
    lines = file.readlines() 
    row = np.zeros(len(supercategory_set))
    for line in lines:
        words = line.strip().split(':')
        supercategory_column = supercategory_dict.get(words[0])
        category_index = category_dict.get(words[1])
        row[supercategory_column] = category_index
    train_tags.append(row)
    file.close()

In [2]:
features_train_concat = np.concatenate((np.array(features_train_2048),np.array(features_train_1000)), axis=1)
features_train_concat = np.concatenate((features_train_concat, np.array(train_tags)), axis=1)

X_train_1 = np.concatenate((xtrain_glove,features_train_concat),axis=1)
Y_train_1 = np.ones(10000)

train_query_reordered = np.concatenate((xtrain_glove[5000:10000],xtrain_glove[0:5000]),axis=0)
X_train_0 = np.concatenate((train_query_reordered,features_train_concat),axis=1)
Y_train_0 = np.zeros(10000)

X_train = np.concatenate((X_train_1,X_train_0),axis=0)
Y_train = np.concatenate((Y_train_1,Y_train_0),axis=0)

In [3]:
features_test_2048 = np.zeros((2000,2048))
with open('./all/data/features_test/features_resnet1000intermediate_test.csv', 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    for line in csv_reader:
        image_name = line[0].strip(".jpg")[12:]
        row = []
        for i in range(len(line)):
            if i > 0:
                row.append(float(line[i]))
        features_test_2048[int(image_name)] = row

features_test_1000 = np.zeros((2000,1000))    
with open('./all/data/features_test/features_resnet1000_test.csv', 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    for line in csv_reader:
        image_name = line[0].strip(".jpg")[12:]
        row = []
        for i in range(len(line)):
            if i > 0:
                row.append(float(line[i]))
        features_test_1000[int(image_name)] = row

# Vectorize test tags
test_tags = []
for i in range (2000):
    file = open("./all/data/tags_test/" + str(i) + ".txt", "r")
    lines = file.readlines() 
    row = np.zeros(len(supercategory_set))
    for line in lines:
        words = line.strip().split(':')
        supercategory_column = supercategory_dict.get(words[0])
        category_index = category_dict.get(words[1])
        row[supercategory_column] = category_index
    test_tags.append(row)
    file.close()
    
features_test_concat = np.concatenate((np.array(features_test_2048),np.array(features_test_1000)), axis=1)
features_test_concat = np.concatenate((features_test_concat, np.array(test_tags)), axis=1)


xtest_glove = np.loadtxt('query_glove_test.csv',delimiter=",", dtype=float)

In [4]:
X_train_random = []
import random
for i in range(len(X_train)):
    X_train_random.append(np.append(X_train[i],np.array([random.randint(0,89) - 9])))
print(X_train.shape)

(20000, 3360)


In [5]:
X_train_random = np.array(X_train_random)
print(X_train_random.shape)

(20000, 3361)


In [6]:
from sklearn.ensemble import RandomForestClassifier
clf_RF = RandomForestClassifier(max_depth=50, random_state=0,n_estimators=200,verbose=3,n_jobs=-1)
clf_RF.fit(X_train_random,Y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


building tree 1 of 200building tree 2 of 200

building tree 3 of 200building tree 4 of 200

building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   14.4s


building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 200
building tree 46 of 200
building tree 47 of 200
building tree 48 of 200
building tree 49 of 200
building tree 50 of 200
building tree 51 of 200
building tree 52 of 200
building tree 53 of 200
building tree 54 of 200
building tree 55 of 200
building tree 56 of 200
building tree 57 of 200
building tree 58 of 200
building tree 59 of 200
building tree 60 of 200
building tree 61 of 200
building tree 62 of 200
building tree 63 of 200
building tree 64 of 200
building tree 65 of 200
building tree 66 of 200
building tree 67 of 200
building tree 68 of 200
building tree 69

[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  1.2min


building tree 124 of 200
building tree 125 of 200
building tree 126 of 200
building tree 127 of 200
building tree 128 of 200
building tree 129 of 200
building tree 130 of 200
building tree 131 of 200
building tree 132 of 200
building tree 133 of 200
building tree 134 of 200
building tree 135 of 200
building tree 136 of 200
building tree 137 of 200
building tree 138 of 200
building tree 139 of 200
building tree 140 of 200
building tree 141 of 200
building tree 142 of 200
building tree 143 of 200
building tree 144 of 200
building tree 145 of 200
building tree 146 of 200
building tree 147 of 200
building tree 148 of 200
building tree 149 of 200
building tree 150 of 200
building tree 151 of 200
building tree 152 of 200
building tree 153 of 200
building tree 154 of 200
building tree 155 of 200
building tree 156 of 200
building tree 157 of 200
building tree 158 of 200
building tree 159 of 200
building tree 160 of 200
building tree 161 of 200
building tree 162 of 200
building tree 163 of 200


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  1.8min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=0, verbose=3, warm_start=False)

In [7]:
importance = clf_RF.feature_importances_
meaningless = importance[-1]
print(meaningless)

features_index = []

for i in range(len(importance) - 1):
    if importance[i] > meaningless:
        features_index.append(i)
        
print (len(features_index))
print (features_index)

0.0004324366809810579
397
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216

In [12]:
X_train_red = X_train[:,features_index]

In [14]:
clf_RF = RandomForestClassifier(max_depth=15, random_state=0,n_estimators=100,verbose=3,n_jobs=-1,max_features=None)
clf_RF.fit(X_train_red,Y_train)

building tree 1 of 100building tree 2 of 100
building tree 3 of 100
building tree 4 of 100

building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.0min


building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  4.4min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=0, verbose=3, warm_start=False)

In [15]:
rank_list = []
for i in tqdm(range (2000)):
    query_repeat = np.tile(xtest_glove[i],(2000,1))
    X_test = np.concatenate((query_repeat,features_test_concat),axis=1)
    X_test_red = X_test[:,features_index]
    svm_predictions = clf_RF.predict_proba(X_test_red)
    predictions_indices = [(svm_predictions[index][1],index) for index in range(2000)]
    predictions_indices.sort(reverse=True)
    rank_list.append([v for (k,v) in predictions_indices[:20]])
def mergeTwenty(lst):
    result = str(lst[0])+".jpg"
    for i in range(1,len(lst)):
        result += " " + str(lst[i]) + ".jpg"
    return result

with open('submission.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    csv_writer.writerow(["Descritpion_ID", "Top_20_Image_IDs"])
    for i in range(len(rank_list)):
        csv_writer.writerow([str(i) + ".txt", mergeTwenty(rank_list[i])])

  0%|          | 0/2000 [00:00<?, ?it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  0%|          | 1/2000 [00:00<08:31,  3.91it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
  0%|          | 2/2000 [00:00<07:49,  4.26it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  0%|          | 3/2000 [00:00<07:00,  4.75it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
  0%|          | 4/2000 [00:00<06:23,  5.20it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
  0%|          | 5/2000 [00:00<06:06,  5.44it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Par

  2%|▏         | 46/2000 [00:07<04:48,  6.77it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
  2%|▏         | 47/2000 [00:07<04:54,  6.64it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  2%|▏         | 48/2000 [00:07<04:50,  6.73it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  2%|▏         | 49/2000 [00:07<04:50,  6.71it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  2%|▎         | 50/2000 [00:07<04:51,  6.69it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  3%|▎         | 51/2000 [00:07<04:51,  6.69it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed

[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  5%|▍         | 92/2000 [00:14<05:02,  6.30it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  5%|▍         | 93/2000 [00:14<05:04,  6.26it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  5%|▍         | 94/2000 [00:14<04:55,  6.44it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  5%|▍         | 95/2000 [00:14<04:48,  6.61it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  5%|▍         | 96/2000 [00:15<04:48,  6.60it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  5%|▍         | 97/2000 [00:1

  7%|▋         | 137/2000 [00:21<04:46,  6.49it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  7%|▋         | 138/2000 [00:21<04:45,  6.53it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  7%|▋         | 139/2000 [00:21<04:47,  6.46it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
  7%|▋         | 140/2000 [00:21<04:42,  6.58it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
  7%|▋         | 141/2000 [00:21<04:48,  6.44it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
  7%|▋         | 142/2000 [00:22<04:54,  6.31it/s][Parallel(n_jobs=4)]: Done  24 tasks      | e

  9%|▉         | 182/2000 [00:28<04:29,  6.74it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  9%|▉         | 183/2000 [00:28<04:29,  6.75it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  9%|▉         | 184/2000 [00:28<04:30,  6.72it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  9%|▉         | 185/2000 [00:28<04:28,  6.77it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  9%|▉         | 186/2000 [00:28<04:26,  6.81it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
  9%|▉         | 187/2000 [00:28<04:24,  6.85it/s][Parallel(n_jobs=4)]: Done  24 tasks      | e

...

 90%|█████████ | 1802/2000 [04:34<00:30,  6.49it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
 90%|█████████ | 1803/2000 [04:34<00:30,  6.52it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
 90%|█████████ | 1804/2000 [04:34<00:31,  6.19it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 90%|█████████ | 1805/2000 [04:34<00:32,  6.07it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
 90%|█████████ | 1806/2000 [04:34<00:31,  6.09it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 90%|█████████ | 1807/2000 [04:34<00:30,  6.24it/s][Parallel(n_jobs=4)]: Done  24 tasks   

 92%|█████████▏| 1847/2000 [04:41<00:22,  6.79it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 92%|█████████▏| 1848/2000 [04:41<00:22,  6.77it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 92%|█████████▏| 1849/2000 [04:41<00:22,  6.68it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 92%|█████████▎| 1850/2000 [04:41<00:22,  6.75it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 93%|█████████▎| 1851/2000 [04:41<00:21,  6.79it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 93%|█████████▎| 1852/2000 [04:41<00:21,  6.83it/s][Parallel(n_jobs=4)]: Done  24 tasks   

 95%|█████████▍| 1892/2000 [04:47<00:15,  6.94it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 95%|█████████▍| 1893/2000 [04:47<00:15,  6.93it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 95%|█████████▍| 1894/2000 [04:47<00:15,  6.87it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 95%|█████████▍| 1895/2000 [04:48<00:15,  6.60it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 95%|█████████▍| 1896/2000 [04:48<00:15,  6.70it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 95%|█████████▍| 1897/2000 [04:48<00:15,  6.74it/s][Parallel(n_jobs=4)]: Done  24 tasks   

 97%|█████████▋| 1937/2000 [04:54<00:09,  6.82it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 97%|█████████▋| 1938/2000 [04:54<00:09,  6.82it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 97%|█████████▋| 1939/2000 [04:54<00:09,  6.76it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 97%|█████████▋| 1940/2000 [04:54<00:08,  6.78it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 97%|█████████▋| 1941/2000 [04:54<00:08,  6.75it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 97%|█████████▋| 1942/2000 [04:54<00:08,  6.80it/s][Parallel(n_jobs=4)]: Done  24 tasks   

 99%|█████████▉| 1982/2000 [05:00<00:02,  6.86it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 99%|█████████▉| 1983/2000 [05:00<00:02,  6.86it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 99%|█████████▉| 1984/2000 [05:01<00:02,  6.85it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 99%|█████████▉| 1985/2000 [05:01<00:02,  6.82it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 99%|█████████▉| 1986/2000 [05:01<00:02,  6.81it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
 99%|█████████▉| 1987/2000 [05:01<00:01,  6.79it/s][Parallel(n_jobs=4)]: Done  24 tasks   

In [16]:
clf_RF = RandomForestClassifier(max_depth=100, random_state=0,n_estimators=500,verbose=3,n_jobs=-1,max_features=None)
clf_RF.fit(X_train_red,Y_train)

rank_list = []
for i in range (2000):
    query_repeat = np.tile(xtest_glove[i],(2000,1))
    X_test = np.concatenate((query_repeat,features_test_concat),axis=1)
    X_test_red = X_test[:,features_index]
    svm_predictions = clf_RF.predict_proba(X_test_red)
    predictions_indices = [(svm_predictions[index][1],index) for index in range(2000)]
    predictions_indices.sort(reverse=True)
    rank_list.append([v for (k,v) in predictions_indices[:20]])
def mergeTwenty(lst):
    result = str(lst[0])+".jpg"
    for i in range(1,len(lst)):
        result += " " + str(lst[i]) + ".jpg"
    return result

with open('submission2.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    csv_writer.writerow(["Descritpion_ID", "Top_20_Image_IDs"])
    for i in range(len(rank_list)):
        csv_writer.writerow([str(i) + ".txt", mergeTwenty(rank_list[i])])

building tree 1 of 500
building tree 2 of 500
building tree 3 of 500building tree 4 of 500

building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.8min


building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500
building tree 44 of 500
building tree 45 of 500
building tree 46 of 500
building tree 47 of 500
building tree 48 of 500
building tree 49 of 500
building tree 50 of 500
building tree 51 of 500
building tree 52 of 500
building tree 53 of 500
building tree 54 of 500
building tree 55 of 500
building tree 56 of 500
building tree 57 of 500
building tree 58 of 500
building tree 59 of 500
building tree 60 of 500
building tree 61 of 500
building tree 62 of 500
building tree 63 of 500
building tree 64 of 500
building tree 65 of 500
building tree 66 of 500
building tree 67 of 500
building tree 68 of 500
building tree 69

[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  8.0min


building tree 125 of 500
building tree 126 of 500
building tree 127 of 500
building tree 128 of 500
building tree 129 of 500
building tree 130 of 500
building tree 131 of 500
building tree 132 of 500
building tree 133 of 500
building tree 134 of 500
building tree 135 of 500
building tree 136 of 500
building tree 137 of 500
building tree 138 of 500
building tree 139 of 500
building tree 140 of 500
building tree 141 of 500
building tree 142 of 500
building tree 143 of 500
building tree 144 of 500
building tree 145 of 500
building tree 146 of 500
building tree 147 of 500
building tree 148 of 500
building tree 149 of 500
building tree 150 of 500
building tree 151 of 500
building tree 152 of 500
building tree 153 of 500
building tree 154 of 500
building tree 155 of 500
building tree 156 of 500
building tree 157 of 500
building tree 158 of 500
building tree 159 of 500
building tree 160 of 500
building tree 161 of 500
building tree 162 of 500
building tree 163 of 500
building tree 164 of 500


...

[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.

[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.

[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.

[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.

[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.

In [18]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

clf_ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=100,max_features=None), random_state=0, n_estimators=500)
clf_ada.fit(X_train_red,Y_train)

rank_list = []
for i in range (2000):
    query_repeat = np.tile(xtest_glove[i],(2000,1))
    X_test = np.concatenate((query_repeat,features_test_concat),axis=1)
    X_test_red = X_test[:,features_index]
    svm_predictions = clf_ada.predict_proba(X_test_red)
    predictions_indices = [(svm_predictions[index][1],index) for index in range(2000)]
    predictions_indices.sort(reverse=True)
    rank_list.append([v for (k,v) in predictions_indices[:20]])
def mergeTwenty(lst):
    result = str(lst[0])+".jpg"
    for i in range(1,len(lst)):
        result += " " + str(lst[i]) + ".jpg"
    return result

with open('submission3.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    csv_writer.writerow(["Descritpion_ID", "Top_20_Image_IDs"])
    for i in range(len(rank_list)):
        csv_writer.writerow([str(i) + ".txt", mergeTwenty(rank_list[i])])

In [20]:
clf_ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), random_state=0, n_estimators=200)
clf_ada.fit(X_train_random,Y_train)


importance = clf_ada.feature_importances_
meaningless = importance[-1]
print(meaningless)

features_index = []

for i in range(len(importance) - 1):
    if importance[i] > meaningless:
        features_index.append(i)
        
print (len(features_index))
print (features_index)






X_train_red = X_train[:,features_index]

clf_ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=100,max_features=None), random_state=0, n_estimators=500)
clf_ada.fit(X_train_red,Y_train)

rank_list = []
for i in range (2000):
    query_repeat = np.tile(xtest_glove[i],(2000,1))
    X_test = np.concatenate((query_repeat,features_test_concat),axis=1)
    X_test_red = X_test[:,features_index]
    svm_predictions = clf_ada.predict_proba(X_test_red)
    predictions_indices = [(svm_predictions[index][1],index) for index in range(2000)]
    predictions_indices.sort(reverse=True)
    rank_list.append([v for (k,v) in predictions_indices[:20]])
def mergeTwenty(lst):
    result = str(lst[0])+".jpg"
    for i in range(1,len(lst)):
        result += " " + str(lst[i]) + ".jpg"
    return result

with open('submission3.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    csv_writer.writerow(["Descritpion_ID", "Top_20_Image_IDs"])
    for i in range(len(rank_list)):
        csv_writer.writerow([str(i) + ".txt", mergeTwenty(rank_list[i])])

1.0
0
[]


ValueError: Found array with 0 feature(s) (shape=(20000, 0)) while a minimum of 1 is required.

In [21]:
print (importance)

[0. 0. 0. ... 0. 0. 1.]


In [22]:
clf_ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), random_state=0, n_estimators=200)
clf_ada.fit(X_train_random,Y_train)

KeyboardInterrupt: 

In [ ]:
print (clf_ada.feature_importances_)